In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 웹사이트
!pip install pyngrok -q
!pip install streamlit -q

# 트랜스포머
!pip install sentencepiece transformers accelerate einops -q
!pip install transformers -q

# 번역
!pip install googletrans==3.1.0a0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.8 MB/s eta 0

In [8]:
# 스트림밋 시작
import streamlit as st
from pyngrok import ngrok

ngrok.set_auth_token('2X0EYF8JF2eQJbF76vX029bQM0F_2MTEtA1HELj7vnJ3NohxC')

In [4]:
%%writefile app.py

import streamlit as st
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from googletrans import Translator

trans = Translator()

my_path = '/content/drive/MyDrive/my_ws/project/Final/phi2'

@st.cache_resource
def load_model_and_tokenizer(my_path):
    torch.set_default_device("cuda")
    model = AutoModelForCausalLM.from_pretrained(my_path, torch_dtype=torch.float32, device_map="cuda", trust_remote_code=True)

    tokenizer = AutoTokenizer.from_pretrained(my_path, trust_remote_code=True)

    return model, tokenizer


model, tokenizer = load_model_and_tokenizer(my_path)

if "temp_count" not in st.session_state:
    st.session_state.temp_count = [0.85, 0.9, 0.95, 1.0]

if "check" not in st.session_state:
    st.session_state.check = False

if "main_story" not in st.session_state:
    st.session_state.main_story = ['군 복무를 마치고 복학한 만섭은 학교에 족구장이 사라졌다는 걸 알게 된다. 그는 이 문제를 해결하기 위해 대학교에 찾아가 족구장을 만들어 달라고 요구한다.',
'최초의 핵무기를 개발한 제2차 세계 대전 프로젝트인 맨해튼 프로젝트에서 역할로 "원자 폭탄의 아버지"로 인정받은 미국의 이론 물리학자 J. 로버트 오펜하이머',
'착하고 예쁜 신데렐라는 비록 귀족의 딸이나, 새엄마의 학대와 배다른 두 언니들의 심술로, 하녀와도 같은 혹독한 나날을 겪고 있다. 그러던 중 요정 대모의 도움으로 신데렐라도 왕궁의 무도회에 갈 수 있게 된다.',
'대한민국을 뒤흔든 10월 26일 이후, 서울에 새로운 바람이 불어온 것도 잠시 12월 12일, 보안사령관 전두광이 반란을 일으키고 군 내 사조직을 총동원하여 최전선의 전방부대까지 서울로 불러들인다.',
'토니 스타크는 뉴욕 사건을 겪은 뒤 슈퍼히어로의 삶에 회의감과 혼란스러움을 느낀다. 그런 그의 앞에 사악한 테러리스트 만다린이 나타나 스타크 저택을 파괴하고 토니의 주변인을 위협한다.'
]
if "main_story_image_url" not in st.session_state:
    st.session_state.main_story_image_url = ['https://i.namu.wiki/i/Bq2x4dxH-r6TlGZxbO-EPwz_Dk0zVfNo9o6n8YUNbzJLi6mBh0Bem4Q6Ww-A_7Srbf5mjSbqaj5LGeQxi-gEMA.webp',
'https://i.namu.wiki/i/rd5oNqqBY3tPw9hF3olu2hhI2Pyi5RwqpfhN_EUwDio-qhmHWa-Vu0VHD4iKk3g1iTbb_a_thcqkiiQSTLmlNQ.webp',
'https://i.namu.wiki/i/7W1qOWViBSDjavzWhlt_peSkrB9Lf2cQ7fEq92DQMLKu3L77347HN0r8zhkXYoriFcnnYoqRvDu97M1CJldW3Q.webp',
'https://i.namu.wiki/i/dQYtrYCT20iBXl8NVB32m-USGBNORan-hsEgwrGpEQeivNd-R-MTfLh0v01lmGV3CUc4PMCuW6BEswVt_Gt6MQ.webp',
'https://i.namu.wiki/i/RdPvwLtJeiOnKyXtjmqugaE6rxp_AlzJdxTi74oPxmr5Zi6w2gKuOHFiNWYayw3GQgukE_fX9CHM0s8FGKjrPQ.webp'
]
if "select_story" not in st.session_state:
    st.session_state.select_story = ['',
'',
'',
''
]
if "selecdo_samplet_story" not in st.session_state:
    st.session_state.do_sample = True

if "main_story_input" not in st.session_state:
    st.session_state.main_story_input = ''

if "result_story" not in st.session_state:
    st.session_state.result_story = ''

if "main_select_index" not in st.session_state:
    st.session_state.main_select_index = 0

if "result_check" not in st.session_state:
    st.session_state.result_check = False

if "sub_title" not in st.session_state:
    st.session_state.sub_title = ['진행하고 하고자 하는 스토리를 선택하여 주세요', '선택지를 선택해주세요']

def generate(prompt: str) -> str:
    result = trans.translate(prompt, dest='en', src='ko')
    prompt = result.text

    inputs = tokenizer(prompt, return_tensors="pt",
                       return_attention_mask=False)
    inputs = {key: value.to("cuda") for key, value in inputs.items()}
    num_input_tokens = inputs['input_ids'].shape[1]

    for i in range(4):
        if(i < 3):
            outputs = model.generate(**inputs, max_length=num_input_tokens+30, do_sample=True, temperature=st.session_state.temp_count[i])

            completion = tokenizer.batch_decode(outputs[:, num_input_tokens:])[0]
            completion_ko = trans.translate(completion.split("<|endoftext|>")[0], dest='ko', src='en').text

            output_split_text(completion_ko, i)
        else:
            inputs = tokenizer(prompt, return_tensors="pt",
                       return_attention_mask=False)
            inputs = {key: value.to("cuda") for key, value in inputs.items()}
            num_input_tokens = inputs['input_ids'].shape[1]
            outputs = model.generate(**inputs, max_length=num_input_tokens+30, do_sample=False, temperature=st.session_state.temp_count[i])

            completion = tokenizer.batch_decode(outputs[:, num_input_tokens:])[0]
            completion_ko = trans.translate(completion.split("<|endoftext|>")[0], dest='ko', src='en').text

            output_split_text(completion_ko, i)


def output_split_text(text: str, index: int):

    # 개행 제거
    text = text.replace('\n', ' ')

    # 마침표를 포함하여 분할
    sentences = re.split('(?<=\.) ', text)

    # 리스트
    st.session_state.select_story[index] = sentences[0]


st.title('시나리오 제작 시뮬레이터')
st.subheader(st.session_state.sub_title[0])

col1, col2, col3, col4, col5 = st.columns(5)


def main_stroy_click(index: int):
    with st.spinner('시나리오 작성중...'):
        st.session_state.main_select_index = index
        st.session_state.main_story_input = st.session_state.main_story[index]
        generate(st.session_state.main_story_input)
        st.session_state.check = True
        st.subheader(st.session_state.sub_title[1])
        st.rerun()

st.write(st.session_state.main_story_input)
if(st.session_state.check == False):
    with col1:
        st.image(st.session_state.main_story_image_url[0])
        if st.button('선택하기', key='button1', use_container_width=True):
            main_stroy_click(0)
    with col2:

        st.image(st.session_state.main_story_image_url[1])
        if st.button('선택하기', key='button2', use_container_width=True):
            main_stroy_click(1)

    with col3:
        st.image(st.session_state.main_story_image_url[2])
        if st.button('선택하기', key='button3', use_container_width=True):
            main_stroy_click(2)

    with col4:
        st.image(st.session_state.main_story_image_url[3])
        if st.button('선택하기', key='button4', use_container_width=True):
            main_stroy_click(3)

    with col5:
        st.image(st.session_state.main_story_image_url[4])
        if st.button('선택하기', key='button5', use_container_width=True):
            main_stroy_click(4)

def select_stroy_click(index: int):
    with st.spinner('시나리오 작성중...'):
        st.session_state.main_story_input += ' '
        st.session_state.main_story_input += st.session_state.select_story[index]
        generate(st.session_state.main_story_input)
        st.rerun()

def result_click(index: int):
    with st.spinner('평점 작성중...'):
        st.session_state.main_story_input += ' '
        st.session_state.main_story_input += st.session_state.select_story[index]
        result_generate(st.session_state.main_story_input)
        st.rerun()

def result_generate(prompt: str) -> str:
    prompt += ' 지금까지의 시나리오의 점수를 1~10점으로 표현해줘'
    result = trans.translate(prompt, dest='en', src='ko')
    prompt = result.text

    inputs = tokenizer(prompt, return_tensors="pt",
                       return_attention_mask=False)
    inputs = {key: value.to("cuda") for key, value in inputs.items()}
    num_input_tokens = inputs['input_ids'].shape[1]

    outputs = model.generate(**inputs, max_length=num_input_tokens+30, do_sample=False, temperature=1)

    completion = tokenizer.batch_decode(outputs[:, num_input_tokens:])[0]
    completion_ko = trans.translate(completion.split("<|endoftext|>")[0], dest='ko', src='en').text

    st.session_state.result_story = completion_ko


if(st.session_state.check and st.session_state.result_check==False):
    if st.button(st.session_state.select_story[0], use_container_width=True):
        select_stroy_click(0)
    if st.button(st.session_state.select_story[1], use_container_width=True):
        select_stroy_click(1)
    if st.button(st.session_state.select_story[2], use_container_width=True):
        select_stroy_click(2)
    if st.button('결론: ' + st.session_state.select_story[3], use_container_width=True):
        st.session_state.result_check = True
        result_click(3)

if(st.session_state.result_check):
    st.image(st.session_state.main_story_image_url[st.session_state.main_select_index])

st.write(st.session_state.result_story)

Writing app.py


In [9]:
!nohup streamlit run app.py --server.port 80 &

nohup: appending output to 'nohup.out'


In [10]:
ngrok.connect(80)

<NgrokTunnel: "https://a6ac-34-125-112-108.ngrok-free.app" -> "http://localhost:80">

In [7]:
ngrok.kill()

In [11]:
!wget -q -O - ipv4.icanhazip.com

34.125.112.108
